In [66]:
%matplotlib inline

import numpy.random as npr
import math
import matplotlib.pyplot as plt

from tqdm import tqdm

In [67]:
import sys
sys.path.append("/home/hofmann/nngame/package/src/rlbase/")
from importlib import reload

import env_gridworld
import misc
import policy
reload(policy)
reload(env_gridworld)

import numpy as np
import tabulate as tb
import misc
reload(misc)

<module 'misc' from '/home/hofmann/nngame/package/src/rlbase/misc.py'>

# Figure 4.1

In [68]:
# Gridworld
env = env_gridworld.GridworldEx41Environment()

pi = policy.UniformPolicy(env.states,env.actions)

_, arr_v = misc.evaluate_policy_iterative(env,pi)

for (i,x) in enumerate(arr_v):
  if i in [1,2,3,10,425]:
    print("Iteration: ",i)
    env.pretty_print(env.reshape(x))

Iteration:  1
--  --  --  --
 0  -1  -1  -1
-1  -1  -1  -1
-1  -1  -1  -1
-1  -1  -1   0
--  --  --  --

Iteration:  2
----  ----  ----  ----
 0    -1.8  -2    -2
-1.8  -2    -2    -2
-2    -2    -2    -1.8
-2    -2    -1.8   0
----  ----  ----  ----

Iteration:  3
----  ----  ----  ----
 0    -2.4  -2.9  -3
-2.4  -2.9  -3    -2.9
-2.9  -3    -2.9  -2.4
-3    -2.9  -2.4   0
----  ----  ----  ----

Iteration:  10
----  ----  ----  ----
 0    -6.1  -8.4  -9
-6.1  -7.7  -8.4  -8.4
-8.4  -8.4  -7.7  -6.1
-9    -8.4  -6.1   0
----  ----  ----  ----

Iteration:  425
---  ---  ---  ---
  0  -14  -20  -22
-14  -18  -20  -20
-20  -20  -18  -14
-22  -20  -14    0
---  ---  ---  ---



# Exercise 4.1
Calculate the q-values for (s,a)=((3,1),"down") and (s,a)=((3,2),"down")

In [69]:
q = misc.get_action_value_function(env,arr_v[-1])
print('q((3,1),"down")',round(q((3,1),"down"),1))
print('q((3,2),"down")',round(q((3,2),"down"),1))

q((3,1),"down") -1.0
q((3,2),"down") -15.0


In [10]:
improved_policy = rl_functions.improve_policy_from_value_function(states,actions,rewards,state_transition,v,gamma)
v = rl_functions.evaluate_policy_linear_system(states,actions,rewards,state_transition,improved_policy,gamma,terminal_states)

print(tb.tabulate(np.round(np.array(v.tolist()+[v[0]]).reshape((4,4)),1)))
improved_policy = rl_functions.improve_policy_from_value_function(states,actions,rewards,state_transition,v,gamma)

tmp = [""]*15
for s in states:
  for a in actions:
    if improved_policy(a,s)>0:
      tmp[s] += a[0]
print(tb.tabulate(np.array(tmp+[tmp[0]]).reshape((4,4))))

AttributeError: module 'misc' has no attribute 'evaluate_policy_linear_system'

In [ ]:
import rl_functions
import numpy as np
import matplotlib.pyplot as plt


# Discount parameter
gamma = 0.9
    
def policy(a,s):
  return (a==0)

# Evaluate policy and visualize value function
v = rl_functions.evaluate_policy_linear_system_two_arg(states,actions,state_transition,policy,gamma)
plt.imshow(v.reshape(nmax+1,nmax+1), cmap='hot', interpolation='nearest',origin='lower')
plt.show()

# Improve policy and visualize policy function
pol1 = rl_functions.improve_policy_from_value_function_two_arg(states, actions, state_transition, v, gamma)
arr = np.zeros((nmax+1,nmax+1))
for i in range(nmax+1):
  for j in range(nmax+1):
    most_probable = 0
    for a in actions:
      if pol1(a,(i,j)) > most_probable:
        arr[i,j] = a
        most_probable = pol1(a,(i,j))
plt.imshow(arr,origin="lower")
plt.colorbar()
